In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import openai
from sentence_transformers import SentenceTransformer, util
import json
from sklearn.metrics import cohen_kappa_score

### extract information from each row

In [3]:
# returns a row of a dataframe in the form of a python list
# In this case, we get the result from one respondent of the survey as a python list
def get_row(file, row_ind):
    df = pd.read_csv(file)
    row = []
    try:
        row = df.iloc[row_ind].tolist()
    except:
        print("************************************")
        print("row index not found in file")
    return row

# row needs to be a list
# extracts the boolean value that indicates whether a restaurant has chinese name or no
def extract_hasCH(row):
    result = []
    for i in range(19,len(row)-3,3):
      result.append(row[i])
    return result

# row needs to be a list
# extracts chinese names of the restaurant
def extract_name(row):
    result = []
    for i in range(20,len(row)-3,3):
      result.append(row[i])
    return result

# row needs to be a list
# extracts image source of the chinese names of the restaurant
def extract_source(row):
    result = []
    for i in range(21,len(row)-3,3):
      result.append(row[i])
    return result

# temp = get_row('Yannis.csv',2)
# extract_name(temp)

# def extract(file, row_ind = 2, info = 'source'):
#     if info == 'name':
#         res = extract_name(get_row(file, row_ind))
#     elif info == 'has_name':
#         res = extract_hasCH(get_row(file, row_ind))
#     else:
#         res = extract_source(get_row(file, row_ind))
#     return res
def extract(file, row_indices):
    result={}
    res_ind = 0
    for ind in row_indices:
        row = get_row(file, ind)
        result[res_ind] = [extract_hasCH(row), extract_name(row), extract_source(row)]
        res_ind += 1
    # getting the starting row index of the data
    result['id'] = int(get_row(file,row_indices[0])[170])
    return result
    
# print(extract('testing.csv',[4,5]))

### Adding extracted info to dataframe

In [3]:
# extracted = extract('testing.csv',[4,5])
def add(file, extracted):
    df = pd.read_csv(file)
#     df.insert(7, "has_name", '')
#     df.insert(8, "CH_name", '')
#     df.insert(9, "source", '')
#     new_df = pd.DataFrame({'has_name':extracted[0][0],'CH_name':extracted[0][1],'source':extracted[0][2]},index=[i for i in range(extracted['id'],extracted['id']+30)])
    new_df = pd.DataFrame({'has_name_1':extracted[0][0],'CH_name_1':extracted[0][1],'source_1':extracted[0][2],'has_name_2':extracted[1][0],'CH_name_2':extracted[1][1],'source_2':extracted[1][2]},index=[i for i in range(extracted['id'],extracted['id']+50)])
    df.update(new_df)
    df.to_csv(file, index=False, encoding='utf-8')
# add('sample.csv',extracted)
    

### Calculating Average Time

In [4]:

def time():
#     df = pd.read_csv(r'time_analysis.csv')
    df = pd.DataFrame()
    df['id']=''
    df['date']=''
    df['duration(seconds)']=''
    df['duration(minutes)']=''
#     for num in range(0,60,30):
    for num in range(420,781,40):
        file = 'annotations/qualtrics_data/batch 2/testing_'+str(num)+' - Sheet1.csv'
        df1 = {'id': get_row(file, 1)[18], 'duration(seconds)': get_row(file, 1)[5], 'duration(minutes)':int(get_row(file, 1)[5])/60,'date': get_row(file, 1)[0]}
        df2 = {'id': get_row(file, 2)[18], 'duration(seconds)': get_row(file, 2)[5], 'duration(minutes)':int(get_row(file, 2)[5])/60,'date': get_row(file, 2)[0]}
#         df1 = {'id': get_row(file, 2)[18], 'duration': get_row(file, 2)[5], 'date': get_row(file, 2)[0]}
#         df2 = {'id': get_row(file, 3)[18], 'duration': get_row(file, 3)[5], 'date': get_row(file, 3)[0]}
        df = df.append(df1, ignore_index = True)
        df = df.append(df2, ignore_index = True)
    df.to_csv('time_analysis_batch2.csv', index=False, encoding='utf-8')
# time()

### Calculating agreemet

In [4]:
def agreement(file):
    # read file and get two answers
    ans1 = get_row(file,1)
    ans2 = get_row(file,2)
    print(len(ans1))
    print(len(ans2))
    print(ans1)
    # loop through answers to get the number of responses that are the same
    # same = 0
    # for ind in range(19,len(ans1)-3,3):
    #     # case 1: no chinese name
    #     if ans1[ind]=='0' and ans2[ind]=='0':
    #         same += 1
    #     # case 2: has Chinese name and chinese names are the same
    #     elif ans1[ind]=='1' and ans2[ind]=='1' and ans1[ind+1]==ans2[ind+1]:
    #         same += 1
    # # return same responses/total responses (40)
    # return same/50
    answers1=[]
    answers2=[]
    for ind in range(19,len(ans1)-3,3):
        answers1.append(ans1[ind])
        answers2.append(ans2[ind])
    return cohen_kappa_score(answers1, answers2)



def calc_agr(begin, end, size,batch,filename):
    df = pd.DataFrame()
    df['file_id']=''
    df['agreement']=''
    for num in range(begin,end,size):
        file = 'annotations/qualtrics_data/batch '+batch+'/testing_'+str(num)+' - Sheet1.csv'
        # df1 = {'file_id': num, 'agreement': agreement(file)}
        df.loc[len(df.index)]=[num, agreement(file)]
        # df = df.append(df1, ignore_index = True)
    df.to_csv(filename, index=False, encoding='utf-8')

calc_agr(0,391,30,'1','agreenment_batch1.csv')
calc_agr(420,781,40,'2','agreenment_batch2.csv')
calc_agr(820,1181,40,'3','agreenment_batch3.csv')
calc_agr(1220,2171,50,'4','agreenment_batch4.csv')
calc_agr(2220,2971,50,'5','agreenment_batch5.csv')
calc_agr(3020,3971,50,'6','agreenment_batch6.csv')
# calc_agr()

112
112
['{"ImportId":"startDate","timeZone":"America/Denver"}', '{"ImportId":"endDate","timeZone":"America/Denver"}', '{"ImportId":"status"}', '{"ImportId":"ipAddress"}', '{"ImportId":"progress"}', '{"ImportId":"duration"}', '{"ImportId":"finished"}', '{"ImportId":"recordedDate","timeZone":"America/Denver"}', '{"ImportId":"_recordId"}', '{"ImportId":"recipientLastName"}', '{"ImportId":"recipientFirstName"}', '{"ImportId":"recipientEmail"}', '{"ImportId":"externalDataReference"}', '{"ImportId":"locationLatitude"}', '{"ImportId":"locationLongitude"}', '{"ImportId":"distributionChannel"}', '{"ImportId":"userLanguage"}', '{"ImportId":"QID16"}', '{"ImportId":"QID14_TEXT"}', '{"ImportId":"1_QID3"}', '{"ImportId":"1_QID3_2_TEXT"}', '{"ImportId":"1_QID11_TEXT"}', '{"ImportId":"2_QID3"}', '{"ImportId":"2_QID3_2_TEXT"}', '{"ImportId":"2_QID11_TEXT"}', '{"ImportId":"3_QID3"}', '{"ImportId":"3_QID3_2_TEXT"}', '{"ImportId":"3_QID11_TEXT"}', '{"ImportId":"4_QID3"}', '{"ImportId":"4_QID3_2_TEXT"}', 

In [5]:
# df_all = pd.concat(['agreenment_batch1.csv','agreenment_batch2.csv','agreenment_batch3.csv','agreenment_batch4.csv','agreenment_batch5.csv','agreenment_batch6.csv'],ignore_index=True)
files = ['agreenment_batch1.csv','agreenment_batch2.csv','agreenment_batch3.csv','agreenment_batch4.csv','agreenment_batch5.csv','agreenment_batch6.csv']
dfs=[]
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)
df_all=pd.concat(dfs,ignore_index=True)
df_all.to_csv('agreement_all.csv',index=False,encoding='utf-8')

print(df_all.loc[:,'agreement'].mean())

0.6730236303971405


### Execution

In [7]:
def main():
    #     for num in range(0,60,30):
    for num in range(3020,3971,50):
        file = 'annotations/qualtrics_data/batch 6/testing_'+str(num)+' - Sheet1.csv'
        extracted = extract(file,[1,2])
# for files 60 and beyond
#         extracted = extract(file,[1,2])
#         extracted = extract(file,[3])
#         print(extracted)
        add('sample.csv',extracted)
main()